In [2]:
import numpy as np
import pandas as pd

import quantutils.dataset.pipeline as ppl
import quantutils.dataset.ml as mlutils
from marketinsights.api.model import MarketInsightsModel
from marketinsights.remote.ml import MIAssembly
from marketinsights.remote.models import MIModelServer

In [3]:
DATASET_ID1 = "4234f0f1b6fcc17f6458696a6cdf5101"  # DOW
MODEL_ID = "3a491b1a-8af6-416d-aa14-f812cbd660bb"

assembly = MIAssembly(MIModelServer(secret="marketinsights-k8s-cred"))
modelConfig = assembly.get_model(MODEL_ID)

# Training Set
dataset, descriptor = assembly.get_dataset_by_id(DATASET_ID1)

# Test Set
dataset2 = pd.read_pickle("testset.pkl")

In [5]:
mlmodel = MarketInsightsModel.createInstance(
    "Perceptron", 
    opts={"name": "MLModel", "featureCount": descriptor["features"], "labelCount": descriptor["labels"], "modelConfig": modelConfig }
)

In [6]:
train_x, train_y = ppl.splitCol(dataset, descriptor["features"])
test_x, test_y = ppl.splitCol(dataset2, descriptor["features"])

In [7]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(train_x).numpy()), ppl.onehot(train_y), threshold=0.5)

Won : 709.0
Lost : 668.0
Total : 1377.0
Diff : 41.0
Edge : 2.9774872912127814%
IR : 1.1048845


0.51488745

In [8]:
mlmodel.restore("../config/modelserver/mlmodel/3")

In [9]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(train_x).numpy()), ppl.onehot(train_y), threshold=0.5)

Won : 1033.0
Lost : 344.0
Total : 1377.0
Diff : 689.0
Edge : 50.03631082062454%
IR : 18.56745


0.75018156

In [11]:
mlutils.evaluate(ppl.onehot(mlmodel.predictions(test_x).numpy()), ppl.onehot(test_y), threshold=0.8)

Won : 33.0
Lost : 17.0
Total : 50.0
Diff : 16.0
Edge : 32.0%
IR : 2.2627416


0.66

In [14]:
import tensorflow as tf
train_x = tf.cast(train_x, tf.float32)
train_y = tf.cast(train_y, tf.float32)
mlmodel.train(train_x, train_y, learning_rate=3.0, epochs=100)

Mean squared error for step 0: 0.481
Precision 0.75
